### Looking at lexically-redacted examples

I'm interested in finding some **fully lexically-redacted examples** that our model identifies correctly.

In [1]:
import sys
sys.path.append('/home/jxm3/research/deidentification/unsupervised-deidentification')

In [2]:
from dataloader import WikipediaDataModule
import os

num_cpus = len(os.sched_getaffinity(0))

dm = WikipediaDataModule(
    document_model_name_or_path="roberta-base",
    profile_model_name_or_path="google/tapas-base",
    max_seq_length=128,
    dataset_name='wiki_bio',
    dataset_train_split='train[:1024]', # not used in this notebook
    dataset_val_split='val[:20%]',
    dataset_version='1.2.0',
    word_dropout_ratio=0.0,
    word_dropout_perc=0.0,
    num_workers=1,
    train_batch_size=64,
    eval_batch_size=64
)
dm.setup("fit")

Initializing WikipediaDataModule with num_workers = 1 and mask token `<mask>`
loading wiki_bio[1.2.0] split train[:1024]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)


loading wiki_bio[1.2.0] split val[:20%]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-793b771e10f80bbe.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-7d07543b6205ca87.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-912d45fbf560a15e.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-4731c171b2d92df3.arrow
Loading cached processed dataset at /h

In [3]:
from model import CoordinateAscentModel
from model_cfg import model_paths_dict

print(model_paths_dict.keys())
checkpoint_path = model_paths_dict["model_8_ls0.1"]


model = CoordinateAscentModel.load_from_checkpoint(
    checkpoint_path
)

dict_keys(['model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8_ls0.01', 'model_8_ls0.05', 'model_8_ls0.1', 'model_9_ls0.01', 'model_9_ls0.05', 'model_9_ls0.1'])


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialized model with learning_rate = 4e-05 and patience 6


In [4]:
import numpy as np
import tqdm
import torch

def precompute_profile_embeddings(model):
    model.profile_model.cuda()
    model.profile_model.eval()
    model.profile_embed.cuda()
    model.profile_embed.eval()

    model.val_profile_embeddings = np.zeros((len(dm.val_dataset), model.shared_embedding_dim))
    for val_batch in tqdm.tqdm(dm.val_dataloader()[0], desc="Precomputing val embeddings", colour="green", leave=False):
        with torch.no_grad():
            profile_embeddings = model.forward_profile(batch=val_batch)
        model.val_profile_embeddings[val_batch["text_key_id"]] = profile_embeddings.cpu()
    model.val_profile_embeddings = torch.tensor(model.val_profile_embeddings, dtype=torch.float32)

precompute_profile_embeddings(model)

In [5]:
from typing import List

import transformers
from model.model import Model

class MyModelWrapper:
    model: Model
    tokenizer: transformers.AutoTokenizer
    profile_embeddings: torch.Tensor
    max_seq_length: int
    
    def __init__(self, model: Model, tokenizer: transformers.AutoTokenizer, max_seq_length: int = 128):
        self.model = model
        self.model.eval()
        self.tokenizer = tokenizer
        self.profile_embeddings = torch.tensor(model.val_profile_embeddings)
        self.max_seq_length = max_seq_length
                 
    def to(self, device):
        self.model.to(device)
        self.profile_embeddings.to(device)
        return self # so semantics `model = MyModelWrapper().to('cuda')` works properly

    def __call__(self, text_input_list: List[str], batch_size=32):
        model_device = next(self.model.parameters()).device
        
        doc_tokenized = self.tokenizer.batch_encode_plus(
            text_input_list,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
        doc_tokenized = {f'document__{k}': v for k,v in doc_tokenized.items()}
        with torch.no_grad():
            document_embeddings = self.model.forward_document(batch=doc_tokenized, document_type='document')
            document_to_profile_logits = document_embeddings @ self.profile_embeddings.T.to(model_device)
            document_to_profile_probs = torch.nn.functional.softmax(
                document_to_profile_logits, dim=-1
            )
        assert document_to_profile_probs.shape == (len(text_input_list), len(self.profile_embeddings))
        return document_to_profile_probs
            

In [6]:
dm.val_dataset

Dataset({
    features: ['input_text', 'target_text', 'name', 'document', 'profile', 'profile_keys', 'profile_values', 'text_key', 'profile__input_ids', 'profile__token_type_ids', 'profile__attention_mask', 'document_redact_lexical', 'document_redact_ner', 'document_redact_idf_20', 'document_redact_idf_40', 'document_redact_idf_60', 'document_redact_idf_80', 'text_key_id'],
    num_rows: 14566
})

In [7]:
model_wrapper = MyModelWrapper(model=model, tokenizer=dm.document_tokenizer)
model_wrapper.to('cuda')

<ipython-input-5-2a86c7eecbdd>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.profile_embeddings = torch.tensor(model.val_profile_embeddings)


In [12]:
i = 0
batch_size = 256
preds = []
while i < len(dm.val_dataset):
    preds.extend(
        model_wrapper(
            dm.val_dataset['document'][i:i+batch_size], batch_size=batch_size
        )
    )
    i += batch_size
preds = torch.stack(preds)

In [27]:
(preds.argmax(dim=-1) == torch.arange(len(dm.val_dataset)).cuda()).float().mean().item()

0.9957435131072998

In [ ]:
i = 0
batch_size = 256
lex_preds = []
while i < len(dm.val_dataset):
    if i % 2048 == 0: print(i)
    lex_preds.extend(
        model_wrapper(
            dm.val_dataset['document_redact_lexical'][i:i+batch_size], batch_size=batch_size
        )
    )
    i += batch_size
lex_preds = torch.stack(lex_preds)

In [26]:
lex_pred_correct = (lex_preds.argmax(dim=-1) == torch.arange(len(dm.val_dataset)).cuda())
lex_pred_correct.float().mean().item()

0.6248111724853516

In [28]:
correct_lex_redacted_docs = [dm.val_dataset[i] for i in range(14566) if lex_pred_correct[i]]
len(correct_lex_redacted_docs)

9101

In [175]:
from IPython.display import HTML, display
import html

wrap_th = lambda s: f'<th>{s}</th>'
wrap_td = lambda s: f'<td>{s}</td>'

def table_from_table_rows(rows_str: str) -> List[Tuple[str, str]]:
    return [[el.strip() for el in r.split('||')] for r in rows_str.split('\n')]

def html_format_doc(doc: str) -> str:
    # return html.escape(doc)
    return doc.replace('<mask>', '<span style="color: darkgray; background-color: darkgray; line-height: 16px; font-size: 14px; font-weight: 100"> &lt;mask&gt;</span>')

def make_prof_html(profile: str) -> str:
    table = table_from_table_rows(profile)
    s = '<table style="border: 1px solid black"><tbody>'
    # print('table:', table)
    for rkey, rval in table:
        s += '<tr>'
        s += f'<th><b>{rkey}</b></th>'
        s += f'<td>{rval}</td>'
        s += '</tr>'
    s += '</tbody></table>'
    return s

table = '<table>'
table += f'<tr>{wrap_th("ID")}{wrap_th("Name")}{wrap_th("Redacted document")}{wrap_th("Profile")}</th>'
for i in range(100):
    table += '<tr>'
    ex = correct_lex_redacted_docs[i]
    table += wrap_td(ex['text_key_id'])
    table += wrap_td(ex['name'])
    table += wrap_td(html_format_doc(ex['document_redact_lexical']))
    table += wrap_td(make_prof_html(ex['profile']))
    table += '</tr>'
table += '</table>'
    
display(HTML(table))

In [33]:
[doc for doc in dm.val_dataset['document'] if 'table tennis' in doc]

['hui jun is a male former table tennis player from china .\n',
 'cornelia molnar ( born 26 november 1983 ) is a croatian table tennis player .\nshe competed for croatia at the 2004 summer olympics and 2012 summer olympics .\n',
 'bora vang ( 王博 , born april 9 , 1987 ) is a chinese-born turkish national table tennis player .\nthe tall athlete at competes for adana table tennis club , where he is coached by sabahattin sabrioğlu .\nhe won the gold medal in the mized doubles event at the 2010 european mixed double championships held in subotica , serbia together with his turkish teammate Şirin he .\nin 2012 , he and his turkish teammate melek hu won the silver medal at the european mixed double championships held in buzau , romania .\nvang qualified for the 2012 summer olympics after the world qualification championship held in doha , qatar .\nat the 2013 mediterranean games held in mersin , turkey , he won the silver medal in the singles event .\n',
 "mikael appelgren ( born october 15 ,

In [59]:
len([doc for doc in dm.val_dataset['document'] if 'brazil' in doc])

207

In [61]:
len([doc for doc in dm.val_dataset['document'] if (('wales' in doc) or ('welsh' in doc)) and ('actor' in doc)])

10

In [72]:
def get_num_words_until_paren(s: str) -> int:
    try:
        return s.split(' ').index('(')
    except ValueError:
        return -1

    
four_word_named_people = [(i, doc) for i,doc in enumerate(dm.val_dataset['document']) if (get_num_words_until_paren(doc) == 4) and lex_pred_correct[i]]
len(four_word_named_people)

579

In [75]:
brazilian_four_word_named_people = [(i, doc) for i, doc in four_word_named_people if (('brazil' in doc) or ('brasil' in doc))]
len(brazilian_four_word_named_people)

26

In [76]:
brazilian_four_word_named_people[0]

(82,
 'thiago dos santos costa ( born february 28 , 1983 ) is a brazilian footballer who plays for são luiz .\n')

In [77]:
brazilian_four_word_named_people[1]

(460,
 'luis ricardo silva umbelino ( born 21 january 1984 ) is a brazilian footballer who plays for botafogo , on loan from são paulo fc .\nmainly a right back , he can perform equally as a right midfielder .\n')

In [78]:
brazilian_four_word_named_people[2]

(493,
 'fabián raphael estay silva ( born october 5 , 1968 ) is a chilean football midfielder .\nhe was capped 69 times and scored five goals for the chile national team between 1990 and 2001 , including four games at the 1998 fifa world cup .\nestay made his debut for the senior national squad on october 17 , 1990 in a friendly against brazil .\nfabián played for club sides in chile , switzerland , greece , mexico and colombia .\n')

In [79]:
brazilian_four_word_named_soccer_players = [(i, doc) for i,doc in brazilian_four_word_named_people if 'play' in doc]
len(brazilian_four_word_named_soccer_players)

19

In [83]:
len([doc for doc in dm.val_dataset['document'] if (('iran' in doc) and ('volleyball' in doc))])

1

In [93]:
import glob
import pandas as pd

adv_csv_path = glob.glob('../adv_csvs/model_8_1day/*')[0]

df = pd.read_csv(adv_csv_path)
df['perturbed_text'] = df['perturbed_text'].map(lambda s: s.replace('<SPLIT>', '\n'))

In [102]:
def make_html_table(rows: List[List[str]]) -> str:
    assert len(rows) > 0
    table = '<table style="font-size: 18px">'
    table += '<tr>'
    table += ''.join([wrap_th(el) for el in rows[0]])
    table += '</tr>'
    for row in rows[1:]:
        table += '<tr>'
        for el in row:
            table += wrap_td(el)
        table += '</tr>'
    table += '</table>'
    return table

In [174]:

def limit_100_words(doc: str) -> str:
    words = doc.split(' ')
    if len(words) > 100:
        doc = ' '.join(words[:100]) + '…'
    return doc

def html_format_doc(doc: str) -> str:
    return doc.replace('<mask>', '<span style="color: lightgray; background-color: lightgray; font-weight: 100"> &lt;mask&gt;</span>')

def display_redacted_doc(idx):
    adv_doc = limit_100_words(df['perturbed_text'][idx])
    lex_doc = limit_100_words(dm.val_dataset[idx]['document_redact_lexical'])
    
    table = make_html_table(
        [
            ['Name', 'Lexically redacted', 'Adversarially redacted'],
            [dm.val_dataset[idx]['name'], html_format_doc(lex_doc), html_format_doc(adv_doc)]
        ]
    )
    
    display(HTML(table))

display_redacted_doc(81)

In [104]:
display_redacted_doc(801)

Name,Lexically redacted,Adversarially redacted
Truid Blaisse-terwindt,"<mask> <mask> ( 4 <mask> <mask> -- <mask> <mask> <mask> ) was a dutch female hockey - and tennis player who was active in the 1930s and 1940s . between 1935 and <mask> she participated in five wimbledon championships . her best result in the singles event was reaching the third round in <mask> , losing to dorothy round , and <mask> , losing to first seeded margaret du pont . in the doubles she reached the third round in <mask> and <mask> partnering compatriot madzy rollin couquerque . with ivo rinkel she reached the fourth round of the mixed doubles in <mask> . in…","<mask> <mask> ( <mask> <mask> <mask> -- <mask> <mask> <mask> ) was <mask> <mask> female <mask> - <mask> tennis player who <mask> <mask> in <mask> 1930s and 1940s . between 1935 and 1948 she participated in five wimbledon championships . her best result in <mask> singles event was reaching <mask> third round in 1937 , <mask> to dorothy round , and 1948 , losing to first <mask> margaret du <mask> . in the doubles she <mask> the third round in 1936 and <mask> <mask> compatriot madzy rollin couquerque . with ivo <mask> she <mask> the fourth round <mask> the mixed doubles in <mask> . in…"


In [145]:
display_redacted_doc(701)

Name,Lexically redacted,Adversarially redacted
Mikhail Bulgakov,"<mask> <mask> <mask> ( ; , ; -- <mask> <mask> , <mask> ) was a <mask> writer and <mask> active in the first half of the 20th century . he is best known for <mask> novel `` the master and margarita '' , which has been called one of the masterpieces of the 20th century .","<mask> <mask> <mask> ( ; , ; -- <mask> 10 , <mask> ) was a russian writer <mask> <mask> active in the first half of <mask> 20th century . he is best known for his <mask> `` the master and margarita '' , which has been called one of the masterpieces of the 20th century ."


In [168]:
import random
display_redacted_doc(random.choice(range(1000)))

Name,Lexically redacted,Adversarially redacted
Tony Do Pilar Patrao,"<mask> <mask> <mask> <mask> ( born <mask> , <mask> in <mask> , ) is a french-portuguese footballer . he currently plays in the cfa for <mask> <mask> . <mask> played at berre l'etang and <mask> . source : http://www.foot-national.com/4168-joueur-football- <mask>- <mask>- <mask>- <mask>.html","<mask> <mask> <mask> <mask> ( born 19 , <mask> <mask> <mask> , ) is a <mask> <mask> . he currently plays in the cfa <mask> <mask> <mask> . <mask> played <mask> berre l'etang and <mask> . source : http:// <mask>.foot-national.com/4168-joueur-football-do-pilar-patrao-tony.html"
